#  **Setup and checks**





In [1]:
import tensorflow as tf
if tf.__version__!="2.0.0-beta1":
  !pip install --upgrade tensorflow-gpu==2.0.0b1
  print("updated Tensorflow library")
import os
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import random
import tensorflow_datasets as tfds


assert tf.__version__=="2.0.0-beta1", "Upgrade tf, then restart runtime"
assert tf.test.is_gpu_available(), "Connect GPU"

keras = tf.keras

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

os.chdir("/content/drive/My Drive/Colab Notebooks/Kaggle/chest_xray_pneumonia")
!pwd &ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/Kaggle/chest_xray_pneumonia
 chest_xray
 chest_xray_dataset
'Copy of ResNet.ipynb'
 Data_exploration.ipynb
 hello_test.txt
 inception_log.txt
 inception_v3_model_2.h5
 inception_v3_model.h5
 main_inception_incremental.ipynb
 main_inception.ipynb
 main.ipynb
 main_ResNet.ipynb
 Models_ensemble.ipynb
 Paper
 Prepare_Kaggle.ipynb
 ResNet.ipynb
 ResNet_log.txt
 ResNet_model.h5
 resnet_train_cache.tf-data_0.data-00000-of-0000



# Load data:
All images will be resized to IMG_SIZE x IMG_SIZE,  and input channels rescaled to a range of [-1,1].


In [2]:
IMG_SIZE = 256 # All images will be resized to this size
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
BATCH_SIZE = 64
num_train, num_val, num_test = 5216, 16, 624
count = 16
SPLIT_PRECETNTAGE = 0.9

DATA_ROOT = "/content/drive/My Drive/Colab Notebooks/Kaggle/chest_xray_pneumonia"
DATA_ROOT = pathlib.Path(DATA_ROOT)


ALL_LOC = pathlib.Path(str(DATA_ROOT) + '/chest_xray/train')
all_imgs_paths = list(ALL_LOC.glob('*/*.jpeg'))
all_imgs_paths = [str(path) for path in all_imgs_paths]
random.shuffle(all_imgs_paths)

label_names = sorted(item.name for item in ALL_LOC.glob('*/') if item.is_dir())
label_to_index = dict((name, index) for index, name in enumerate(label_names))

all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                    for path in all_imgs_paths]
""
image_count = len(all_imgs_paths)
split_edge = round(image_count * SPLIT_PRECETNTAGE)

train_paths, train_labels = all_imgs_paths[:split_edge], all_image_labels[:split_edge]
val_paths, val_labels =all_imgs_paths[split_edge:], all_image_labels[split_edge:] 

print(len(train_paths), len(val_paths), len(all_imgs_paths))

for i in range(5):
  print(train_labels[i], train_paths[i])

####################################################################################################
TEST_LOC = pathlib.Path(str(DATA_ROOT) + '/chest_xray/test')
test_imgs_paths = list(TEST_LOC.glob('*/*.jpeg'))
test_imgs_paths = [str(path) for path in test_imgs_paths]
random.shuffle(test_imgs_paths)

test_label_names = sorted(item.name for item in TEST_LOC.glob('*/') if item.is_dir())
test_label_to_index = dict((name, index) for index, name in enumerate(test_label_names))

test_image_labels = [test_label_to_index[pathlib.Path(path).parent.name]
                    for path in test_imgs_paths]
""
test_image_count = len(test_imgs_paths)

test_paths, test_labels = test_imgs_paths, test_image_labels

print("\nTrain images are %d image, but Test images are %d." % (len(train_paths), len(test_paths)))


for i in range(5):
  print(test_labels[i], test_paths[i])


4694 522 5216
1 /content/drive/My Drive/Colab Notebooks/Kaggle/chest_xray_pneumonia/chest_xray/train/PNEUMONIA/person149_virus_299.jpeg
1 /content/drive/My Drive/Colab Notebooks/Kaggle/chest_xray_pneumonia/chest_xray/train/PNEUMONIA/person1396_virus_2399.jpeg
1 /content/drive/My Drive/Colab Notebooks/Kaggle/chest_xray_pneumonia/chest_xray/train/PNEUMONIA/person26_bacteria_131.jpeg
1 /content/drive/My Drive/Colab Notebooks/Kaggle/chest_xray_pneumonia/chest_xray/train/PNEUMONIA/person257_bacteria_1193.jpeg
1 /content/drive/My Drive/Colab Notebooks/Kaggle/chest_xray_pneumonia/chest_xray/train/PNEUMONIA/person458_bacteria_1953.jpeg

Train images are 4694 image, but Test images are 624.
1 /content/drive/My Drive/Colab Notebooks/Kaggle/chest_xray_pneumonia/chest_xray/test/PNEUMONIA/person89_bacteria_440.jpeg
0 /content/drive/My Drive/Colab Notebooks/Kaggle/chest_xray_pneumonia/chest_xray/test/NORMAL/NORMAL2-IM-0246-0001-0001.jpeg
1 /content/drive/My Drive/Colab Notebooks/Kaggle/chest_xray_pn

In [0]:
def preprocess_image(image):
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
  image = image/127.5 -1 # normalize to [-1,1] range

  return image

def load_and_preprocess_image(path):
  image = tf.io.read_file(path)
  return preprocess_image(image)

def load_and_preprocess_from_path_label(path, label):
  return load_and_preprocess_image(path), label

def show_patient(img, label):
  img = img.numpy()
  print(img.shape)
  fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 15))
  fig.suptitle(label_names[label].title(), fontsize=24)
#   plt.subplots_adjust(hspace=0.1, top=0.1, bottom=0.05)
  for i, ax in enumerate(axs.flat):
      ax.imshow(img[...,i], "gray")
  plt.show()

In [0]:
""" Building a dataset """


ds_train = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
train_ds = ds_train.map(load_and_preprocess_from_path_label)

train_ds = train_ds.shuffle(buffer_size=500)
train_ds = train_ds.repeat()
training_batches = train_ds.batch(BATCH_SIZE)

ds_val = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))
val_ds = ds_val.map(load_and_preprocess_from_path_label)

validation_batches = val_ds.cache() # filename='./resnet_val_cache.tf-data'
validation_batches = validation_batches.batch(BATCH_SIZE)#.prefetch(1)

ds_test = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))
test_ds = ds_test.map(load_and_preprocess_from_path_label)

test_batches = test_ds.batch(BATCH_SIZE)#.prefetch(1)



# > **Now loading both models:**

1.   Model 1: Inception v3
2.   Model 2: ResNet




In [5]:
model_1 = tf.keras.models.load_model('./inception_v3_model_2.h5')
model_1.summary()

W0802 21:50:36.258234 140164683728768 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 6, 6, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               262272    
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 258       
Total params: 22,065,314
Trainable params: 21,692,514
Non-trainable params: 372,800
_________________________________________________________________


In [8]:
model_1.layers[-1]

In [0]:
new_model = tf.keras.Sequential(
                              model_1.layers,
#                               model_1.layers[-1],
                              )
new_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 6, 6, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               262272    
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 258       
Total params: 22,065,314
Trainable params: 21,692,514
Non-trainable params: 372,800
_________________________________________________________________


In [0]:
for x, y in test_batches.take(1):
  y_pred1 = model_1.get_output_at(-1)
  print("1: ", str(y_pred1))
#   tf.print(y_pred1, output_stream=sys.stdout)
  y_newmodel = new_model.predict_classes(x)
  print("2: ", y_newmodel)

1:  Tensor("dense_3/Identity:0", shape=(None, 2), dtype=float32)
2:  [0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0
 1 1 0 1 0 1 0 1 0 0 1 0 0 1 1 0 0 1 0 1 0 0 1 0 0 0 0]


In [0]:
model_1.evaluate(test_batches, steps = 5)

5/5 [==============================] - 122s 24s/step - loss: 1.5448 - accuracy: 0.8016


[1.5448121547698974, 0.8015625]

In [0]:
for x, y in test_batches.take(4):
  y_pred = model_1.predict_classes(x)
  con_mat1 = tf.math.confusion_matrix(labels=y, predictions=y_pred).numpy()
  
print(con_mat1)

[[16  7]
 [24 17]]


In [0]:
print(y[2])

tf.Tensor(1, shape=(), dtype=int32)


In [0]:
model_2 = tf.keras.models.load_model('./ResNet_model.h5')
model_2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 8, 8, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 23,524,802
Non-trainable params: 67,008
_________________________________________________________________


In [0]:
model_2.evaluate(test_batches, steps = 5)

5/5 [==============================] - 8s 2s/step - loss: 6.0375 - accuracy: 0.6062


[6.0374627113342285, 0.60625]

In [0]:
for x, y in test_batches.take(4):
  y_pred = model_2.predict_classes(x)
  con_mat2 = tf.math.confusion_matrix(labels=y, predictions=y_pred).numpy()
  
print(con_mat2)

[[23  0]
 [41  0]]


In [0]:
'''
model_2.compile(loss='binary_crossentropy',
              optimizer = tf.keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])

for i in range(5):
  fine_tune_epochs = 50 #* (i+1)
  no_epochs = 450 + 50 * (i)
  total_epochs =  no_epochs + fine_tune_epochs
  history_fine = model_2.fit(training_batches,
                         epochs=total_epochs,
                         initial_epoch = no_epochs,
                         steps_per_epoch = 5,
                         validation_data = validation_batches,
                         verbose=2)
  model_2.save('./ResNet_model.h5')
  
  print("\n\nFinished loop nmber %d\nwith Total epochs of %d\n\n"% (i, total_epochs))

'''

'\nmodel_2.compile(loss=\'binary_crossentropy\',\n              optimizer = tf.keras.optimizers.Adam(lr=0.0001),\n              metrics=[\'accuracy\'])\n\nfor i in range(5):\n  fine_tune_epochs = 50 #* (i+1)\n  no_epochs = 450 + 50 * (i)\n  total_epochs =  no_epochs + fine_tune_epochs\n  history_fine = model_2.fit(training_batches,\n                         epochs=total_epochs,\n                         initial_epoch = no_epochs,\n                         steps_per_epoch = 5,\n                         validation_data = validation_batches,\n                         verbose=2)\n  model_2.save(\'./ResNet_model.h5\')\n  \n  print("\n\nFinished loop nmber %d\nwith Total epochs of %d\n\n"% (i, total_epochs))\n\n'

In [0]:
# model_2.evaluate(test_batches, steps = 5)

In [0]:
# for x, y in test_batches.take(4):
#   y_pred = model_2.predict_classes(x)
#   con_mat2 = tf.math.confusion_matrix(labels=y, predictions=y_pred).numpy()
  
# print(con_mat2)

In [0]:
w1 = 0.83
w2 = 1 - w1

for x, y_true in test_batches.take(1):
  y1_pred=model_1.predict_classes(x)
  y2_pred=model_2.predict_classes(x)

  y_pred = w1*y1_pred + w2*y2_pred

  con_mat = tf.math.confusion_matrix(labels=y_true, predictions=y_pred).numpy()

In [0]:
print(con_mat)

[[26  0]
 [38  0]]
